In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#data_root = "../sample_data/competitive-data-science-predict-future-sales/"

train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [4]:
#We are asking you to predict total sales for every product and store in the next month.
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()
train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
train = train.query('item_cnt_day >= 0 and item_cnt_day <= 125 and item_price < 75000')
#train = train.drop('date', axis=1)
#train = train.drop('date_block_num', axis=1)
train.head()

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id,month,year
0,0,01.09.2014,20,54,58.0,1.0,40,09,2014
1,1,04.04.2014,15,55,4490.0,1.0,76,04,2014
2,1,02.04.2014,15,55,4490.0,1.0,76,04,2014
3,1,06.07.2014,18,55,4490.0,1.0,76,07,2014
4,1,04.08.2014,19,55,4490.0,1.0,76,08,2014


In [5]:
train_m=train.groupby(["date_block_num","shop_id","item_id"])["date_block_num","date","item_price","item_cnt_day"].agg(
    {"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

train_m.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


date_block_num        date              \
                                         mean         min         max   
date_block_num shop_id item_id                                          
0              0       32                 0.0  03.01.2013  31.01.2013   
                       33                 0.0  03.01.2013  28.01.2013   
                       35                 0.0  31.01.2013  31.01.2013   
                       43                 0.0  31.01.2013  31.01.2013   
                       51                 0.0  13.01.2013  31.01.2013   

                               item_price item_cnt_day  
                                     mean          sum  
date_block_num shop_id item_id                          
0              0       32           221.0          6.0  
                       33           347.0          3.0  
                       35           247.0          1.0  
                       43           221.0          1.0  
                       51           128.5          2.0

In [6]:
train_flat = train_m.item_cnt_day.apply(list).reset_index()
train_flat = pd.merge(test,train_flat,on = ['item_id','shop_id'],how = 'left')
train_flat.fillna(0,inplace = True)
train_flat.drop(['shop_id','item_id'],inplace = True, axis = 1)
train_flat.head(20)

,ID,date_block_num,sum
0,0,20.0,1.0
1,0,22.0,1.0
2,0,23.0,2.0
3,0,24.0,2.0
4,0,28.0,1.0
5,0,29.0,1.0
6,0,30.0,1.0
7,0,31.0,3.0
8,0,32.0,1.0
9,1,0.0,0.0


In [7]:
pivoted_train = train_flat.pivot_table(index='ID', columns='date_block_num',fill_value = 0,aggfunc='sum' )
pivoted_train.head(2)

sum                                               ...       \
date_block_num 0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0   ... 24.0   
ID                                                                ...        
0                 0    0    0    0    0    0    0    0    0    0  ...    2   
1                 0    0    0    0    0    0    0    0    0    0  ...    0   

                                                             
date_block_num 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0  
ID                                                           
0                 0    0    0    1    1    1    3    1    0  
1                 0    0    0    0    0    0    0    0    0  

[2 rows x 34 columns]

In [9]:
X_train = train_flat[["ID", "date_block_num"]]
y_train = train_flat[["sum"]]

X_train_p = np.expand_dims(pivoted_train.values[:,:-1],axis = 2)
y_train_p = pivoted_train.values[:,-1:]
X_test_p = np.expand_dims(pivoted_train.values[:,1:],axis = 2)

X_train.shape

(702705, 2)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

lstm_model = Sequential()
lstm_model.add(LSTM(units = 64,input_shape = (33,1)))
lstm_model.add(Dropout(0.4))
lstm_model.add(Dense(1))

lstm_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.models import Sequential
from keras.layers import Input,Dense,Dropout
from keras.models import load_model, Model

other_model = Sequential()
other_model.add(Input((33,1)))
other_model.add(Dense(64, activation="sigmoid"))
other_model.add(Dropout(0.5))
other_model.add(Dense(32, activation="relu"))
other_model.add(Dense(1))

other_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
other_model.summary()
     

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 33, 64)            128       
                                                                 
 dropout_1 (Dropout)         (None, 33, 64)            0         
                                                                 
 dense_2 (Dense)             (None, 33, 32)            2080      
                                                                 
 dense_3 (Dense)             (None, 33, 1)             33        
                                                                 
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________


In [12]:


lstm_model.fit(X_train_p,y_train_p,batch_size = 4096,epochs = 20)
     


Epoch 1/20
53/53 [==============================] - 23s 378ms/step - loss: 5.7420 - mean_squared_error: 5.7420
Epoch 2/20
53/53 [==============================] - 20s 376ms/step - loss: 5.4803 - mean_squared_error: 5.4803
Epoch 3/20
53/53 [==============================] - 21s 387ms/step - loss: 5.3282 - mean_squared_error: 5.3282
Epoch 4/20
53/53 [==============================] - 20s 384ms/step - loss: 5.2237 - mean_squared_error: 5.2237
Epoch 5/20
53/53 [==============================] - 21s 391ms/step - loss: 5.1862 - mean_squared_error: 5.1862
Epoch 6/20
53/53 [==============================] - 20s 378ms/step - loss: 5.1099 - mean_squared_error: 5.1099
Epoch 7/20
53/53 [==============================] - 21s 389ms/step - loss: 5.0677 - mean_squared_error: 5.0677
Epoch 8/20
53/53 [==============================] - 21s 392ms/step - loss: 5.0530 - mean_squared_error: 5.0530
Epoch 9/20
53/53 [==============================] - 21s 399ms/step - loss: 4.9882 - mean_squared_error: 4.9882
E

In [13]:
other_model.fit(X_train_p,y_train_p,batch_size = 4096,epochs = 20)
     

Epoch 1/20
53/53 [==============================] - 11s 182ms/step - loss: 6.2049 - mean_squared_error: 6.2049
Epoch 2/20
53/53 [==============================] - 10s 182ms/step - loss: 6.0120 - mean_squared_error: 6.0120
Epoch 3/20
53/53 [==============================] - 10s 181ms/step - loss: 5.9545 - mean_squared_error: 5.9545
Epoch 4/20
53/53 [==============================] - 10s 186ms/step - loss: 5.9128 - mean_squared_error: 5.9128
Epoch 5/20
53/53 [==============================] - 10s 182ms/step - loss: 5.8705 - mean_squared_error: 5.8705
Epoch 6/20
53/53 [==============================] - 10s 184ms/step - loss: 5.8282 - mean_squared_error: 5.8282
Epoch 7/20
53/53 [==============================] - 10s 191ms/step - loss: 5.7858 - mean_squared_error: 5.7858
Epoch 8/20
53/53 [==============================] - 10s 183ms/step - loss: 5.7315 - mean_squared_error: 5.7315
Epoch 9/20
53/53 [==============================] - 10s 185ms/step - loss: 5.6804 - mean_squared_error: 5.6804
E

In [17]:
#submission_output = other_model.predict(X_test_p)
submission_output = lstm_model.predict(X_test_p)
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_output.ravel()})

submission.to_csv('/kaggle/working/submission_stacked.csv',index = False)
submission.head()



6694/6694 [==============================] - 44s 7ms/step


,ID,item_cnt_month
0,0,0.618183
1,1,0.161382
2,2,0.856154
3,3,0.214171
4,4,0.161382
